In [1]:
import random
import numpy as np
import numpy.matlib
import pandas as pd

feature_i = pd.read_csv("p2/feature_i.csv", index_col=0)
feature_ii = pd.read_csv("p2/feature_ii.csv", index_col=0)
feature_ii.columns = "ii_"+feature_ii.columns
feature_iii = pd.read_csv("p2/feature_iii.csv", index_col=0)
feature_iii.columns = "iii_"+feature_iii.columns
feature_iv = pd.read_csv("p2/feature_iv.csv", index_col=0)
feature_iv.columns = "iv_"+feature_iv.columns
feature_v = pd.read_csv("p2/feature_v.csv", index_col=0)
feature_v.columns = "v_"+feature_v.columns
feature_vi = pd.read_csv("p2/feature_vi.csv", index_col=0)
new_feature = pd.concat([feature_i, feature_ii, feature_iii], axis=1)
new_feature = new_feature.loc[new_feature["day"]>=28,]
new_feature = pd.concat([new_feature, feature_iv, feature_v], axis=1)
new_feature = pd.merge(new_feature, feature_vi)
new_feature.sort_values(["day", "pluno"], inplace = True)
new_feature = new_feature.reset_index()
del new_feature["index"]
del new_feature["ii_pluno"]
del new_feature["iii_pluno"]
del new_feature["iv_pluno"]
del new_feature["v_pluno"]

new_feature

,pluno,bndno,type5,type4,type3,type2,day,weekday,qty,d-1,...,4min-w-4,5ave-w-2,5ave-w-3,5ave-w-4,5max-w-2,5max-w-3,5max-w-4,5min-w-2,5min-w-3,5min-w-4
0,22000005,22000005,22000,2200,220,22,28,1,0.0,0.0,...,0.001,0.117333,0.000000,0.000000,0.704,0.000,0.000,0.0,0.0,0.0
1,22000008,22000008,22000,2200,220,22,28,1,0.0,0.0,...,0.001,0.117333,0.000000,0.000000,0.704,0.000,0.000,0.0,0.0,0.0
2,22000009,22000009,22000,2200,220,22,28,1,0.0,0.0,...,0.001,0.117333,0.000000,0.000000,0.704,0.000,0.000,0.0,0.0,0.0
3,22000010,22000010,22000,2200,220,22,28,1,0.0,0.0,...,0.001,0.117333,0.000000,0.000000,0.704,0.000,0.000,0.0,0.0,0.0
4,22000014,22000014,22000,2200,220,22,28,1,0.0,0.0,...,0.001,0.117333,0.000000,0.000000,0.704,0.000,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93935,27410042,27410042,27410,2741,274,27,181,0,0.0,0.0,...,0.000,0.675667,6.392167,4.342167,3.008,19.012,12.014,0.0,0.0,0.0
93936,27410043,27410043,27410,2741,274,27,181,0,0.0,0.0,...,0.000,0.675667,6.392167,4.342167,3.008,19.012,12.014,0.0,0.0,0.0
93937,27410044,27410044,27410,2741,274,27,181,0,0.0,0.0,...,0.000,0.675667,6.392167,4.342167,3.008,19.012,12.014,0.0,0.0,0.0
93938,27410045,27410045,27410,2741,274,27,181,0,0.0,0.0,...,0.000,0.675667,6.392167,4.342167,3.008,19.012,12.014,0.0,0.0,0.0


In [3]:
feature_name = new_feature.columns.values.tolist()
feature_name.remove("qty")
day = 18*7+28-1
# feature_name

In [4]:
x_i = new_feature.loc[(new_feature["day"] <= day),feature_name].values
y_i = new_feature.loc[(new_feature["day"] <= day),"qty"].values
x_i = np.reshape(x_i, (-1, len(feature_name)))
y_i = np.reshape(y_i, (-1, 1))

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_i = sc_X.fit_transform(x_i)
Y_i = sc_y.fit_transform(y_i)

In [5]:
import time
time_list = [[0,0],[0,0],[0,0]]

from sklearn.svm import SVR
time_start=time.time()
regressor_i = SVR(kernel = 'rbf')
regressor_i.fit(X_i, Y_i)
time_end=time.time()
time_list[0][0] = time_end - time_start

test_i = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
del test_i["index"]
test_i["predict"] = test_i["qty"]
time_start=time.time()
for i, row in test_i.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i.loc[i, "d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["pluno"]==int(row["pluno"])), "predict"].sum()
        test_i.loc[i, "ii_d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["bndno"]==int(row["bndno"])), "predict"].sum()
        test_i.loc[i, "iii_5d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["type5"]==int(row["type5"])), "predict"].sum()
        test_i.loc[i, "iii_4d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["type4"]==int(row["type4"])), "predict"].sum()
        test_i.loc[i, "iii_3d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["type3"]==int(row["type3"])), "predict"].sum()
        test_i.loc[i, "iii_2d-"+str(j)] = test_i.loc[(test_i["day"]==int(row["day"])-j) & (test_i["type2"]==int(row["type2"])), "predict"].sum()
        j = j+1
#     for week_index in range(2, 5):
#         week_2 = test_i.loc[(test_i["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i["day"]>=int(row["day"])-7*week_index)&(test_i["pluno"]==int(row["pluno"])), "predict"].values
#         test_i.loc[i,"iv_ave-w-"+str(week_index)] = np.mean(week_2)
#         test_i.loc[i,"iv_min-w-"+str(week_index)] = min(week_2)
#         test_i.loc[i,"iv_max-w-"+str(week_index)] = max(week_2)
#         week_2_b_tmp = test_i.loc[(test_i["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i["day"]>=int(row["day"])-7*week_index)&(test_i["bndno"]==int(row["bndno"])),["predict", "bndno", "day"]]
#         week_2_b = week_2_b_tmp["predict"].groupby(week_2_b_tmp['day']).sum().values
#         test_i.loc[i,"v_ave-w-"+str(week_index)] = np.mean(week_2_b)
#         test_i.loc[i,"v_min-w-"+str(week_index)] = min(week_2_b)
#         test_i.loc[i,"v_max-w-"+str(week_index)] = max(week_2_b)
#         for type_index in range(2, 6):
#             week_2_5_tmp = test_i.loc[(test_i["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i["day"]>=int(row["day"])-7*week_index)&(test_i["type"+str(type_index)]==int(row["type"+str(type_index)])),["predict", "type"+str(type_index), "day"]]
#             week_2_5 = week_2_5_tmp["predict"].groupby(week_2_5_tmp['day']).sum().values
#             test_i.loc[i,str(type_index)+"ave-w-"+str(week_index)] = np.mean(week_2_5)
#             test_i.loc[i,str(type_index)+"min-w-"+str(week_index)] = min(week_2_5)
#             test_i.loc[i,str(type_index)+"max-w-"+str(week_index)] = max(week_2_5)
    y_pred = regressor_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i

time_end=time.time()
time_list[0][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# i. random forest
from sklearn.ensemble import RandomForestRegressor 
time_start=time.time()
rfr_i =RandomForestRegressor()
rfr_i.fit(X_i, Y_i)
time_end=time.time()
time_list[1][0] = time_end - time_start

test_i_rfr = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
del test_i_rfr["index"]
test_i_rfr["predict"] = test_i_rfr["qty"]
time_start=time.time()
for i, row in test_i_rfr.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i_rfr.loc[i, "d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["pluno"]==int(row["pluno"])), "predict"].sum()
        test_i_rfr.loc[i, "ii_d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["bndno"]==int(row["bndno"])), "predict"].sum()
        test_i_rfr.loc[i, "iii_5d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["type5"]==int(row["type5"])), "predict"].sum()
        test_i_rfr.loc[i, "iii_4d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["type4"]==int(row["type4"])), "predict"].sum()
        test_i_rfr.loc[i, "iii_3d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["type3"]==int(row["type3"])), "predict"].sum()
        test_i_rfr.loc[i, "iii_2d-"+str(j)] = test_i_rfr.loc[(test_i_rfr["day"]==int(row["day"])-j) & (test_i_rfr["type2"]==int(row["type2"])), "predict"].sum()
        j = j+1
#     for week_index in range(2, 5):
#         week_2 = test_i_rfr.loc[(test_i_rfr["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_rfr["day"]>=int(row["day"])-7*week_index)&(test_i_rfr["pluno"]==int(row["pluno"])), "predict"].values
#         test_i_rfr.loc[i,"iv_ave-w-"+str(week_index)] = np.mean(week_2)
#         test_i_rfr.loc[i,"iv_min-w-"+str(week_index)] = min(week_2)
#         test_i_rfr.loc[i,"iv_max-w-"+str(week_index)] = max(week_2)
#         week_2_b_tmp = test_i_rfr.loc[(test_i_rfr["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_rfr["day"]>=int(row["day"])-7*week_index)&(test_i_rfr["bndno"]==int(row["bndno"])),["predict", "bndno", "day"]]
#         week_2_b = week_2_b_tmp["predict"].groupby(week_2_b_tmp['day']).sum().values
#         test_i_rfr.loc[i,"v_ave-w-"+str(week_index)] = np.mean(week_2_b)
#         test_i_rfr.loc[i,"v_min-w-"+str(week_index)] = min(week_2_b)
#         test_i_rfr.loc[i,"v_max-w-"+str(week_index)] = max(week_2_b)
#         for type_index in range(2, 6):
#             week_2_5_tmp = test_i_rfr.loc[(test_i_rfr["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_rfr["day"]>=int(row["day"])-14)&(test_i_rfr["type"+str(type_index)]==int(row["type"+str(type_index)])),["predict", "type"+str(type_index), "day"]]
#             week_2_5 = week_2_5_tmp["predict"].groupby(week_2_5_tmp['day']).sum().values
#             test_i_rfr.loc[i,str(type_index)+"ave-w-"+str(week_index)] = np.mean(week_2_5)
#             test_i_rfr.loc[i,str(type_index)+"min-w-"+str(week_index)] = min(week_2_5)
#             test_i_rfr.loc[i,str(type_index)+"max-w-"+str(week_index)] = max(week_2_5)
    y_pred = rfr_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i_rfr.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_rfr

time_end=time.time()
time_list[1][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [7]:
# i. MLP
from sklearn.neural_network import MLPRegressor
time_start=time.time()
mlp_i = MLPRegressor(
        hidden_layer_sizes=(100,50), activation='relu',solver='adam',
        alpha=0.001,max_iter=200)
# mlp_i = MLPRegressor()
mlp_i.fit(X_i,Y_i)
time_end=time.time()
time_list[2][0] = time_end - time_start

test_i_mlp = new_feature.loc[(new_feature["day"] > day-28),:].reset_index()
del test_i_mlp["index"]
test_i_mlp["predict"] = test_i_mlp["qty"]
time_start=time.time()
for i, row in test_i_mlp.iterrows():
    if row["day"] <= day:
        continue
    j = 1
    while j <= row["day"]%7:
        test_i_mlp.loc[i, "d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["pluno"]==int(row["pluno"])), "predict"].sum()
        test_i_mlp.loc[i, "ii_d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["bndno"]==int(row["bndno"])), "predict"].sum()
        test_i_mlp.loc[i, "iii_5d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["type5"]==int(row["type5"])), "predict"].sum()
        test_i_mlp.loc[i, "iii_4d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["type4"]==int(row["type4"])), "predict"].sum()
        test_i_mlp.loc[i, "iii_3d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["type3"]==int(row["type3"])), "predict"].sum()
        test_i_mlp.loc[i, "iii_2d-"+str(j)] = test_i_mlp.loc[(test_i_mlp["day"]==int(row["day"])-j) & (test_i_mlp["type2"]==int(row["type2"])), "predict"].sum()
        j = j+1
#     for week_index in range(2, 5):
#         week_2 = test_i_mlp.loc[(test_i_mlp["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_mlp["day"]>=int(row["day"])-7*week_index)&(test_i_mlp["pluno"]==int(row["pluno"])), "predict"].values
#         test_i_mlp.loc[i,"iv_ave-w-"+str(week_index)] = np.mean(week_2)
#         test_i_mlp.loc[i,"iv_min-w-"+str(week_index)] = min(week_2)
#         test_i_mlp.loc[i,"iv_max-w-"+str(week_index)] = max(week_2)
#         week_2_b_tmp = test_i_mlp.loc[(test_i_mlp["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_mlp["day"]>=int(row["day"])-7*week_index)&(test_i_mlp["bndno"]==int(row["bndno"])),["predict", "bndno", "day"]]
#         week_2_b = week_2_b_tmp["predict"].groupby(week_2_b_tmp['day']).sum().values
#         test_i_mlp.loc[i,"v_ave-w-"+str(week_index)] = np.mean(week_2_b)
#         test_i_mlp.loc[i,"v_min-w-"+str(week_index)] = min(week_2_b)
#         test_i_mlp.loc[i,"v_max-w-"+str(week_index)] = max(week_2_b)
#         for type_index in range(2, 6):
#             week_2_5_tmp = test_i_mlp.loc[(test_i_mlp["day"]<=int(row["day"])-(7*(week_index-1)+1))&(test_i_mlp["day"]>=int(row["day"])-7*week_index)&(test_i_mlp["type"+str(type_index)]==int(row["type"+str(type_index)])),["predict", "type"+str(type_index), "day"]]
#             week_2_5 = week_2_5_tmp["predict"].groupby(week_2_5_tmp['day']).sum().values
#             test_i_mlp.loc[i,str(type_index)+"ave-w-"+str(week_index)] = np.mean(week_2_5)
#             test_i_mlp.loc[i,str(type_index)+"min-w-"+str(week_index)] = min(week_2_5)
#             test_i_mlp.loc[i,str(type_index)+"max-w-"+str(week_index)] = max(week_2_5)
    y_pred = mlp_i.predict(
        sc_X.transform(
            np.reshape(
            row[feature_name].values
            , (-1, len(feature_name)))))
    test_i_mlp.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_mlp

time_end=time.time()
time_list[2][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
import math

def getRSE(data):
    avg = np.mean(data["qty"])
    return math.sqrt((data["qty"]-data["predict"]).apply(lambda x:math.pow(x, 2)).sum()/data["qty"].apply(lambda x:math.pow(x-avg, 2)).sum())

In [15]:
hat_svr = test_i.loc[:, ["qty", "predict", "day"]]
hat_svr.to_csv("2p3/4/svr.csv")
RSE_svr = getRSE(hat_svr)
RSE_svr

0.6556691307190992

In [16]:
hat_rfr = test_i_rfr.loc[:, ["qty", "predict", "day"]]
hat_rfr.to_csv("2p3/4/rfr.csv")
RSE_rfr = getRSE(hat_rfr)
RSE_rfr

0.5830222098913406

In [17]:
hat_mlp = test_i_mlp.loc[:, ["qty", "predict", "day"]]
hat_mlp.to_csv("2p3/4/mlp.csv")
RSE_mlp = getRSE(hat_mlp)
RSE_mlp

0.5720325517341787

In [18]:
horizon = range(1, 8)
results = [hat_svr, hat_rfr, hat_mlp]
RES_results = []
for re in results:
    ls = []
    for i in horizon:
        ls.append(getRSE(re.loc[re["day"]%7==(i-1), :]))
    RES_results.append(ls)
RES_results

[[0.9401125345179211,
  0.5716038655961306,
  0.48962426910088036,
  0.4621990134077155,
  0.5080119058679273,
  0.8027667759616205,
  0.8977822788004007],
 [0.45987967680544634,
  0.7142064234623329,
  0.38477304234793713,
  0.6950620650948268,
  0.5787319852483758,
  0.5966110880874649,
  0.894029344120625],
 [0.6931254388097412,
  0.4549222564570274,
  0.3778125310496247,
  0.4352170100473241,
  0.5206352291605907,
  0.8117228296318183,
  0.8972965611971304]]

In [14]:
time_list

[[41.74859666824341, 275.83200907707214],
 [479.854660987854, 347.86184763908386],
 [35.92172813415527, 281.8718931674957]]